# my voki tester notebook

In [17]:
import datetime
import numpy as np
import pandas as pd
import os
from pathlib import Path
import pickle
import sys
import mysql.connector
import sqlite3
from sqlite3 import Error
from hashlib import md5
import pyvoki as vo
import random

## settings

In [10]:
# go to root
root = Path('C:/coding/voki/')
os.chdir(root)
assert os.getcwd().endswith('voki'), f'The cwd is not "voki" but {os.getcwd()}'

# make jupyter reload the package
%load_ext autoreload
%autoreload 2

# set the seed for reproducable results
np.random.seed(123)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
N_LEVELS = 6
MAX_BOX_CARDS = 20

db_raw_fpath = root / 'data/voki_export_full_db.csv'
db_raw = pd.read_csv(db_raw_fpath,sep=';')
db_raw['book']=db_raw['file'].str.slice(start=31, stop=32).astype(int)
db_raw.head(1)
data_dir = root/ 'data'
trainer_dir = root / 'db/v1/'

In [13]:
trainer = vo.VokiTrainer(db_raw,
                          trainer_dir,
                          n_levels=N_LEVELS,
                          max_box_cards=MAX_BOX_CARDS)
trainer

Voki Trainer with 3637 entries in db

In [14]:
cards = [vo.Card(row) for i, row in trainer.db.sample(30).iterrows()]
cards[:2]

[Card "une fille" with ID c7829a07c0a25fbf39b8f9a9a7b6858b,
 Card "le ciel" with ID 2c210d593e30ca2a31c400317ca120fd]

In [18]:
for card in cards[:MAX_BOX_CARDS - 1]:
    level = random.randint(1, 6)
    trainer.box.onboard_card(card, level=level)

level 1
    Ça va. 2B Es geht (mir) gut.
    un champ
level 2
    le ciel
    gros/grosse
    Niki de Saint-Phalle
level 3
    sage
    le tabac
    se rappeler qc
    le Canada
    le vieux Clermont
    une fleur
    Non.
    les conditions (f ., pl.) de vie
level 4
    marron (inv.)
    agréable
level 5
level 6
    une fille
    la peinture
    partir pour Paris
    jaloux (-ouse).

In [35]:
trainer.box.onboard_card(cards[0],level=3)

2022-02-19 19:57:05.267 | WARNING  | pyvoki.main:check_card_already_in_box:205 - Card Card "le foot(ball)" with ID 800e1971a4cb02ad700575ff791530aa is already in this box at level 3


False

In [10]:
a=vo.Card(db.db.loc['9e947a5ead01d5e6627aaff10825e46c'])

In [4]:
# some testing
card_1 = pd.DataFrame({'french_phrase': ['partout'],
                            'french_sentences': [['Dans la rue du Sentier, la mode est partout.']],
                            'french_comments': [[]],
                            'german_phrase': ['überall'],
                            'german_sentences': [[]],
                            'german_comments': [[]],
                            'created': [datetime.datetime.now().replace(microsecond=0)],
                            'attempt_history': [pd.DataFrame({'time': pd.Series(dtype='datetime64[ns]'),
                                                             'result': pd.Series(dtype='bool'),
                                                             'answer': pd.Series(dtype='str')})],
                            'streak': [0],
                            'id': [hash('partout')]
                            })

card_2 = pd.DataFrame({'french_phrase': ['la rue'],
                            'french_sentences': [['Dans la rue du Sentier, la mode est partout.']],
                            'french_comments': [[]],
                            'german_phrase': ['Strasse'],
                            'german_sentences': [[]],
                            'german_comments': [[]],
                            'created': [datetime.datetime.now().replace(microsecond=0)],
                            'attempt_history': [pd.DataFrame({'time': pd.Series(dtype='datetime64[ns]'),
                                                             'result': pd.Series(dtype='bool'),
                                                             'answer': pd.Series(dtype='str')})],
                            'streak': [0],
                            'id': [hash('la rue')]
                            })
df = pd.concat([card_1,card_2])
myvokibox = vokibox(5)
myvokibox.onboard_card(card_1,level=4)
myvokibox.onboard_card(card_2)
myvokibox

level 0
    4588279920212162422
level 1
level 2
level 3
level 4
    -2048216720741500272

In [213]:
myvokibox.attempt_card_in_level(4,df);

überall partout


correct!
congrats! card "partout" made it through the vokibox!


In [189]:
myvokibox.attempt_card_in_level(0,df)

überall partout


correct!


In [170]:
myvokibox

level 0
    442165381189763710
level 1
level 2
level 3
level 4
level 5
    -4980411815809341467

In [165]:
myvokibox.attempt_card_in_level(1,df)

vokibox lvl 1 is empty. you need to fill it first


In [166]:
myvokibox.attempt_card_in_level(2,df)

überall partout


correct!


In [167]:
myvokibox.attempt_card_in_level(3,df)

überall partout


correct!


In [169]:
myvokibox.attempt_card_in_level(4,df)

überall partout


correct!


In [171]:
myvokibox.attempt_card_in_level(5,df)

überall partout


correct!
congrats! card partout made it through thevokibox!


In [172]:
myvokibox

level 0
    442165381189763710
level 1
level 2
level 3
level 4
level 5

In [140]:
class card:
    # a class for vocabulary cards

    def __init__(self, props):
        # the basic card that has typical properties of a vocabulary card
        self.french_phrase = props['french']['phrase']
        self.french_sentences =      props['french']['sentences']
        self.french_comments = props['french']['comments']
        self.german_phrase = props['german']['phrase']
        self.german_sentences = props['german']['sentences']
        self.german_comments = props['german']['comments']

        self.created = datetime.datetime.now().replace(microsecond=0)
        self.attempt_history = pd.DataFrame({'time': pd.Series(dtype='datetime64[ns]'),
                                             'result': pd.Series(dtype='bool'),
                                             'answer': pd.Series(dtype='str')})
        self.id = hash(self.french_phrase)
        self.streak = 0

    def attempt(self, guess):
        now = datetime.datetime.now().replace(microsecond=0)
        result = self.check_attempt(guess) #true/False

        self.attempt_history = self.attempt_history.append(pd.DataFrame({'time': [now], 
                                                                        'result': [result], 
                                                                        'answer': [guess]}))
        if result:
            self.streak += 1
        else:
            self.streak = np.floor(self.streak/2)

    def check_attempt(self,answer):
        if answer == self.french_phrase:
            return True
        text = input(f'type "yes" if you think they match:\n {answer} \n {self.french_phrase}\n')
        if text == 'yes':
            return True
        return False
    
    def __repr__(self):
        
        if len(self.attempt_history)>0:
            last_attempt = ''
        str_ = f"""voki card 
        {self.id}
        french: {self.french_phrase}
        german: {self.german_phrase} 
        last_attempt: """
        return str_
    
    


    @staticmethod
    def get_card_from_db(db,id):
        pass
    

        
    



        

        
def main(dbfile):
    #1 laod database
    #2 run program
    #3 save databas
    pass



In [185]:
mycard = card_from_id(4980411815809341467,df)

IndexError: single positional indexer is out-of-bounds

In [177]:
mycard

{'french_phrase': 'la rue',
 'french_sentences': ['Dans la rue du Sentier, la mode est partout.'],
 'french_comments': [],
 'german_phrase': 'Strasse',
 'german_sentences': [],
 'german_comments': [],
 'created': Timestamp('2021-10-31 21:20:50'),
 'attempt_history': Empty DataFrame
 Columns: [time, result, answer]
 Index: [],
 'streak': 0,
 'id': 442165381189763710}

In [179]:
pd.DataFrame({key: [value] for key,value in mycard.items()})

,french_phrase,french_sentences,french_comments,german_phrase,german_sentences,german_comments,created,attempt_history,streak,id
0,la rue,"[Dans la rue du Sentier, la mode est partout.]",[],Strasse,[],[],2021-10-31 21:20:50,"Empty DataFrame Columns: [time, result, answer...",0,442165381189763710


In [170]:
root = Path.cwd()
data_dir = root.parent / 'data'
voki_db_file = data_dir / 'voki_db.pkl'
box_db_file = data_dir / 'box_db.pkl'

# main()

# df_voki = pd.read_pickle(voki_db_file)
# voki_box = box_db_load(box_db_file)

In [161]:
voki_box

level 0
    5116320665103601376
    463857425755945451
level 1
level 2
level 3
level 4
level 5

In [162]:
df_voki

,french_phrase,french_sentences,french_comments,german_phrase,german_sentences,created,attempt_history,id,streak
0,partout,"[Dans la rue du Sentier, la mode est partout.]",[],überall,[],2021-10-31 13:48:03,"Empty DataFrame Columns: [time, result, answer...",463857425755945451,0
0,la rue,[Julien habite dans la rue Daguerre.],[],Strasse,[],2021-10-31 13:48:04,"Empty DataFrame Columns: [time, result, answer...",5116320665103601376,0


In [115]:
my_vbox.onboard_card(voki_card1.id)
my_vbox

level 0
    5116320665103601376
    463857425755945451
level 1
level 2
level 3
level 4
level 5

In [121]:
box_db_load(box_db_file)

level 0
    5116320665103601376
    463857425755945451
level 1
level 2
level 3
level 4
level 5

In [3]:
card_1_dict = {'french_phrase': 'partout', 
                'french_sentences': ['Dans la rue du Sentier, la mode est partout.'],
                'french_comments':[],
                'german_phrase' :'überall',
                'german_sentences':[],
                'german_comments':[],
                'created': datetime.datetime.now().replace(microsecond=0),
                'attempt_history' : pd.DataFrame({'time': pd.Series(dtype='datetime64[ns]'),
                                             'result': pd.Series(dtype='bool'),
                                             'answer': pd.Series(dtype='str')})
                'streak': 0
                'id': hash('partout')
              }
                     
card_2_properties = {'french':{'phrase': 'la rue', 
                               'sentences': ['Julien habite dans la rue Daguerre.'],
                               'comments':[]},
                     'german':{'phrase':'Strasse',
                               'sentences':[],
                               'comments':[]},
                    }   

In [168]:
a=[[]]
a

[[]]

In [4]:
voki_card1=card(card_1_properties)
voki_card1

voki card 
        463857425755945451
        french: partout
        german: überall 
        last_attempt: 

In [5]:
voki_card2=card(card_2_properties)
voki_card2

voki card 
        5116320665103601376
        french: la rue
        german: Strasse 
        last_attempt: 

In [91]:
voki_card1.attempt('partout2'                  )


type "yes" if you think they match:
 partout2 
 partout no


In [92]:
voki_card1.attempt_history

,time,result,answer
0,2021-10-31 08:58:40,1.0,partout
0,2021-10-31 09:17:37,1.0,partout
0,2021-10-31 09:17:40,0.0,partout2


In [16]:
pd.DataFrame(voki_card1.to_df())

TypeError: to_df() got an unexpected keyword argument 'index'

In [19]:
df_list = [pd.DataFrame(voki_card1.to_df()),
           pd.DataFrame(voki_card2.to_df())]
df_voki = pd.concat(df_list)

In [32]:
df_voki.to_pickle(voki_db_file)

In [26]:
df_voki

,french_phrase,french_sentences,french_comments,german_phrase,german_sentences,created,attempt_history,id,streak
0,partout,"[Dans la rue du Sentier, la mode est partout.]",[],überall,[],2021-10-31 13:48:03,"Empty DataFrame Columns: [time, result, answer...",463857425755945451,0
0,la rue,[Julien habite dans la rue Daguerre.],[],Strasse,[],2021-10-31 13:48:04,"Empty DataFrame Columns: [time, result, answer...",5116320665103601376,0


In [35]:
a=pd.read_pickle(voki_db_file)
a

,french_phrase,french_sentences,french_comments,german_phrase,german_sentences,created,attempt_history,id,streak
0,partout,"[Dans la rue du Sentier, la mode est partout.]",[],überall,[],2021-10-31 13:48:03,"Empty DataFrame Columns: [time, result, answer...",463857425755945451,0
0,la rue,[Julien habite dans la rue Daguerre.],[],Strasse,[],2021-10-31 13:48:04,"Empty DataFrame Columns: [time, result, answer...",5116320665103601376,0
